# HMDA Cleaner

The purpose of this program is to clean HMDA data further data needs.

This program is designed to be used for future data HMDA publications.

## Import HMDA Data, Packages, and Check Dataset.

In [6]:
#This code does not need to be changed.
#Imports packages to be used by program
import pandas as pd
import numpy as np

In [7]:
#This code does not need to be changed.
#Quality of Life settings
#This will allow all columns to be displayed when reviewing the data.
pd.options.display.max_columns = None

In [8]:
#Loads in Sample
HMDA_dataset_file_location = r'2019_HMDA_sample.csv'

In [19]:
#This code does not need to be changed.
HMDA_dataset = pd.read_csv(HMDA_sample_file_location)
HMDA_dataset

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-1,applicant_ethnicity-2,applicant_ethnicity-3,applicant_ethnicity-4,applicant_ethnicity-5,co-applicant_ethnicity-1,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,applicant_sex,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,applicant_age,co-applicant_age,applicant_age_above_62,co-applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2019,5493000YNV8IX4VD3X12,99999,KY,21187.0,2.118797e+10,C,Conventional:First Lien,Single Family (1-4 Units):Manufactured,Not Hispanic or Latino,White,Female,5,0,2,1,1,1,2,2,2,95000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,276,NaN,NaN,2,2,2,2,NaN,2,1,1,1,1,NaN,32.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2,2,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,2,2,2,2,2,25-34,45-54,No,No,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,3297,5.58,49800,135,1007,1471,31
1,2019,549300O6Z0I6KYMESL47,16984,IL,17031.0,1.703181e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Joint,4,0,2,1,1,1,2,2,1,215000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,360.0,NaN,NaN,2,1,2,2,NaN,1,3,3,5,1,NaN,0.0,NaN,9,9,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2,2,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,2,1,2,2,2,55-64,>74,No,Yes,2,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,4279,39.10,82000,112,1331,1484,57
2,2019,549300XRXBA38J60S618,99999,ID,16039.0,1.603996e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Male,1,0,2,1,1,1,2,2,2,345000.0,90.0,4.375,0.462,2,4633.0,NaN,1595.0,NaN,NaN,360,NaN,NaN,2,2,2,2,385000,1,1,3,5,1,NaN,0.0,44,3,10,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2,2,5.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,2,2,1,1,2,2,55-64,65-74,Yes,Yes,1,1,1,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,2570,22.49,57100,89,854,1916,39
3,2019,549300NBFJM2PBH24L77,48620,KS,20173.0,2.017301e+10,C,Conventional:First Lien,Single Family (1-4 Units):Manufactured,Not Hispanic or Latino,White,Male,1,0,2,1,31,1,1111,1111,1111,15000.0,Exempt,Exempt,Exempt,3,Exempt,Exempt,Exempt,Exempt,Exempt,Exempt,Exempt,Exempt,1111,1111,1111,1111,Exempt,2,3,1111,1111,1,Exempt,85.0,Exempt,1111,1111,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,1,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,1,4,1,5,1,4,25-34,9999,No,NaN,1111,1111,1111,NaN,NaN,NaN,NaN,1111,NaN,NaN,NaN,5346,20.50,69500,126,1667,1904,24
4,2019,C5654JQHZUHN0772B561,47894,VA,51683.0,5.168391e+10,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Male

In [21]:
HMDA_dataset.columns

Index(['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code',
       'census_tract', 'conforming_loan_limit', 'derived_loan_product_type',
       'derived_dwelling_category', 'derived_ethnicity', 'derived_race',
       'derived_sex', 'action_taken', 'purchaser_type', 'preapproval',
       'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage',
       'open-end_line_of_credit', 'business_or_commercial_purpose',
       'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread',
       'hoepa_status', 'total_loan_costs', 'total_points_and_fees',
       'origination_charges', 'discount_points', 'lender_credits', 'loan_term',
       'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization',
       'interest_only_payment', 'balloon_payment',
       'other_nonamortizing_features', 'property_value', 'construction_method',
       'occupancy_type', 'manufactured_home_secured_property_type',
       'manufactured_home_land_property_interest', 'total_

In [141]:
#This code does not need to be changed unless column headers change.(Ex. state_code changes to state)
#Program to call for future HDMA data.
def HMDA_cleaner(HMDA_dataset_file_location):
    #load in dataset.
    HMDA_dataset = pd.read_csv(HMDA_dataset_file_location)
    
    #Clean Race data.
    df1 = HMDA_dataset.replace(to_replace = {'applicant_race-1' : {
                                (2,21,22,23,24,25,26,27):'Asian',
                                  (3):'Black',
                                 (5):'White',
                                 (1, 4,41,42,43,44):'Other',
                                 (6,7, np.nan):'Not Reported'}})
    #Remove 'Not Reported' rows.
    df2 = df1[df1['applicant_race-1'] != 'Not Reported']
    
    #Clean Ethnicity Data.
    df3 = df2.replace(to_replace = {'applicant_ethnicity-1' : {
                                (1,11,14,12,13):'Hispanic',
                                  (2):'Not Hispanic',
                                 (3,4,np.nan):'Not Reported',}})
    #Remove 'Not Reported' rows.
    df4 = df3[df3['applicant_ethnicity-1'] != 'Not Reported']
    
    #Get dummy variables for applicant race and ethnicity.
    df5 = pd.get_dummies(df4, columns = ['applicant_race-1', 'applicant_ethnicity-1'])
    
    #Remove rows that had no income reported or an income less than 0.
    def income_reported_to_categorical(table, column):
        values = table[column].values
        ir_array =  []
        for i in values:
            if i == np.nan:
                ir_array.append('No Report')
            elif i < 0:
                ir_array.append('Zero or Negative')
            elif i >= 0:
                ir_array.append('Reported')
            else:
                ir_array.append('No Report')
            
        table['Income Reported'] = ir_array     
        return table
    
    df6 = income_reported_to_categorical(df5,'income')
    df7 = df6[df6["Income Reported"].str.contains("Reported") == True]
    
    #Create approval indicator column.
    #Preapproval request denied could also be used as an indicator variable if need be.
    def action_taken_to_approval_indicator(table, column):
        values = table[column].values
        ir_array =  []
        for i in values:
            if i == 1:
                ir_array.append('Approved')
            elif i == 2:
                ir_array.append('Approved')
            elif i == 3:
                ir_array.append('Denied')
            else:
                ir_array.append('other')
            
        table2 = table.copy()
        table2['Approval Indicator'] = ir_array
        
        table3 = table2[table2["Approval Indicator"].str.contains("other") == False]
        
        table4 = pd.get_dummies(table3, columns = ['Approval Indicator'])
        
        return table4
    df8 = action_taken_to_approval_indicator(df7, 'action_taken')
    
    #Clean applicant gender
    df9 = df8.replace(to_replace = {'applicant_sex' : {
                                (1):'Male',
                                  (2):'Female'}})
    df10 = pd.get_dummies(df9, columns = ['applicant_sex'])
   
    #Rename columns
    dfrename = df10.rename(columns = {'Approval Indicator_Approved' : 'Approved',
                                       'Approval Indicator_Denied' : 'Denied',
                                        'income': 'Income',
                                        'applicant_race-1_Asian': 'Asian',
                                        'applicant_race-1_Black': 'Black',
                                        'applicant_race-1_Other': 'Other',
                                        'applicant_race-1_White': 'White',
                                        'applicant_ethnicity-1_Hispanic': 'Hispanic',
                                        'applicant_ethnicity-1_Not Hispanic': 'Not Hispanic',
                                        'applicant_sex_Female': 'Female',
                                        'applicant_sex_Male': 'Male'
                                      })    
    
    
    return dfrename

In [142]:
#Run HMDA_cleaner.
#Enter file location in the HMDA_cleaner(r'file_location') format.
HMDA_cleaner(r'2019_HMDA_sample.csv')

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,Income,debt_to_income_ratio,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-2,applicant_ethnicity-3,applicant_ethnicity-4,applicant_ethnicity-5,co-applicant_ethnicity-1,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,applicant_age,co-applicant_age,applicant_age_above_62,co-applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,Asian,Black,Other,White,Hispanic,Not Hispanic,Income Reported,Approved,Denied,Female,Male
2,2019,549300XRXBA38J60S618,99999,ID,16039.0,1.603996e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Male,1,0,2,1,1,1,2,2,2,345000.0,90.0,4.375,0.462,2,4633.0,NaN,1595.0,NaN,NaN,360,NaN,NaN,2,2,2,2,385000,1,1,3,5,1,NaN,0.0,44,3,10,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,2,2,1,2,2,55-64,65-74,Yes,Yes,1,1,1,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,2570,22.49,57100,89,854,1916,39,0,0,0,1,0,1,Reported,1,0,0,1
3,2019,549300NBFJM2PBH24L77,48620,KS,20173.0,2.017301e+10,C,Conventional:First Lien,Single Family (1-4 Units):Manufactured,Not Hispanic or Latino,White,Male,1,0,2,1,31,1,1111,1111,1111,15000.0,Exempt,Exempt,Exempt,3,Exempt,Exempt,Exempt,Exempt,Exempt,Exempt,Exempt,Exempt,1111,1111,1111,1111,Exempt,2,3,1111,1111,1,Exempt,85.0,Exempt,1111,1111,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,1,4,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,1,4,5,1,4,25-34,9999,No,NaN,1111,1111,1111,NaN,NaN,NaN,NaN,1111,NaN,NaN,NaN,5346,20.50,69500,126,1667,1904,24,0,0,0,1,0,1,Reported,1,0,0,1
4,2019,C5654JQHZUHN0772B561,47894,VA,51683.0,5.168391e+10,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Male,1,2,2,3,1,1,2,2,2,395000.0,100.0,3.5,-0.1828,2,11566.57,NaN,497.98,497.98,NaN,360,NaN,NaN,2,2,2,2,395000.0,1,1,3,5,1,NaN,115.0,30%-<36%,1,10,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,5,2,4,25-34,9999,No,NaN,1,1,1,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,5854,51.81,114700,76,1025,1355,29,0,0,0,1,0,1,Reported,1,0,0,1
7,2019,KB1H1DSPRFMYMCUFXT09,99999,NM,35041.0,3.504100e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Joint,White,Joint,1,3,2,1,32,1,2,2,2,175000.0,63.603,3.625,0.455,2,4017.93,NaN,1662.5,432.5,NaN,180,NaN,NaN,2,2,2,2,275000.0,1,1,3,5,1,NaN,82.0,20%-<30%,1,9,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,2,2,2,2,65-74,55-64,Yes,No,1,1,2,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,4631,38.03,53100,84,611,1599,40,0,0,0,1,0,1,Reported,1,0,0,1
8